In [42]:
from urllib.parse import quote
import requests
import json
import urllib
import xlwt

In [47]:
amap_web_key = 'a24568050fb7dc805c042df45a28dba5'
poi_search_url = "http://restapi.amap.com/v3/place/text"
city = '北京'

In [48]:
i=0
all_poi=[]
while True:
    req_url = poi_search_url + "?key=" + amap_web_key + '&extensions=all&types=141201'  + '&city=' + quote(city) + '&citylimit=true' +'&children=1'+ '&offset=25' + '&page=' + str(
            i) + '&output=json'
    data = ''
    with urllib.request.urlopen(req_url) as f:
        data = f.read()
        data = data.decode('utf-8')
        result=json.loads(data)
        poilist=result['pois']
        all_poi.extend(poilist)
    i = i + 1
        
        

KeyError: 'pois'

In [33]:
n=poilist[0]['parent']
type(n)

list

In [35]:
poilist2=[]
for i in range(len(poilist)):
    if poilist[i]['parent']==[]:
        poilist2.append(poilist[i])
print(poilist2)
    

[{'id': 'B000A816R6', 'parent': [], 'childtype': [], 'name': '北京大学', 'type': '科教文化服务;学校;高等院校', 'typecode': '141201', 'biz_type': [], 'address': '颐和园路5号', 'location': '116.310905,39.992806', 'tel': '010-62752114', 'distance': [], 'biz_ext': [], 'pname': '北京市', 'cityname': '北京市', 'adname': '海淀区', 'importance': [], 'shopid': [], 'shopinfo': '0', 'poiweight': [], 'photos': []}, {'id': 'B000A7BD6C', 'parent': [], 'childtype': [], 'name': '清华大学', 'type': '科教文化服务;学校;高等院校', 'typecode': '141201', 'biz_type': [], 'address': '双清路30号', 'location': '116.326836,40.00366', 'tel': '010-62793001', 'distance': [], 'biz_ext': [], 'pname': '北京市', 'cityname': '北京市', 'adname': '海淀区', 'importance': [], 'shopid': [], 'shopinfo': '0', 'poiweight': [], 'photos': []}, {'id': 'B000A7C66K', 'parent': [], 'childtype': [], 'name': '对外经济贸易大学', 'type': '科教文化服务;学校;高等院校', 'typecode': '141201', 'biz_type': [], 'address': '惠新东街10号', 'location': '116.427745,39.980363', 'tel': '010-64492178;010-64492001', 'distance': [], 'b

In [1]:
from urllib.parse import quote
from urllib import request
import json
import xlwt

#TODO 替换为上面申请的密钥
amap_web_key = 'a24568050fb7dc805c042df45a28dba5'
poi_search_url = "http://restapi.amap.com/v3/place/text"
poi_boundary_url = "https://ditu.amap.com/detail/get/detail"
#from transCoordinateSystem import gcj02_to_wgs84

#TODO cityname为需要爬取的POI所属的城市名，nanning_areas为城市下面的所有区，classes为多个分类名集合. (中文名或者代码都可以，代码详见高德地图的POI分类编码表)
cityname = '北京'

classes = ['141201','141205','141206']


# 根据城市名称和分类关键字获取poi数据
def getpois(cityname, keywords):
    i = 1
    poilist = []
    while True:  # 使用while循环不断分页获取数据
        result = getpoi_page(cityname, keywords, i)
        print(result)
        result = json.loads(result)  # 将字符串转换为json
        if result['count'] == '0':
            break
        hand(poilist, result)
        i = i + 1
    return poilist


# 数据写入excel
def write_to_excel(poilist, cityname, classfield):
    # 一个Workbook对象，这就相当于创建了一个Excel文件
    book = xlwt.Workbook(encoding='utf-8', style_compression=0)
    sheet = book.add_sheet(classfield, cell_overwrite_ok=True)

    # 第一行(列标题)
    sheet.write(0, 0, 'x')
    sheet.write(0, 1, 'y')
    sheet.write(0, 2, 'address')
    sheet.write(0, 3, 'name')
    sheet.write(0, 4, 'adname')


    for i in range(len(poilist)):
        location = poilist[i]['location']
        name = poilist[i]['name']
        address=poilist[i]['address']
        adname = poilist[i]['adname']
        lng = str(location).split(",")[0]
        lat = str(location).split(",")[1]

        '''
        result = gcj02_to_wgs84(float(lng), float(lat))

        lng = result[0]
        lat = result[1]
        '''

        # 每一行写入
        sheet.write(i + 1, 0, lng)
        sheet.write(i + 1, 1, lat)
        sheet.write(i + 1, 2, address)
        sheet.write(i + 1, 3, name)
        sheet.write(i + 1, 4, adname)



    # 最后，将以上操作保存到指定的Excel文件中
    book.save(r'' + cityname + "_" + classfield + '.xls')


# 将返回的poi数据装入集合返回
def hand(poilist, result):
    # result = json.loads(result)  # 将字符串转换为json
    pois = result['pois']
    for i in range(len(pois)):
        poilist.append(pois[i])


# 单页获取pois
def getpoi_page(cityname, keywords, page):
    req_url = poi_search_url + "?key=" + amap_web_key + '&extensions=all&types=' + quote(
        keywords) + '&city=' + quote(cityname) + '&citylimit=true' + '&offset=25' + '&page=' + str(
        page) + '&output=json'
    data = ''
    with request.urlopen(req_url) as f:
        data = f.read()
        data = data.decode('utf-8')
    return data


for clas in classes:
    classes_all_pois = []
    pois_area = getpois(cityname
                        , clas)
    print('分类：' + str(clas) + ", 总的有" + str(len(pois_area)) + "条数据")
    classes_all_pois.extend(pois_area)
    
    classes_all_pois2=[]
    for i in range(len(classes_all_pois)):
        if classes_all_pois[i]['parent']==[]:
            classes_all_pois2.append(classes_all_pois[i])


    write_to_excel(classes_all_pois2, cityname, clas)

    print('================分类：'  + str(clas) + "写入成功")



{"status":"1","count":"895","info":"OK","infocode":"10000","suggestion":{"keywords":[],"cities":[]},"pois":[{"id":"B000A816R6","parent":[],"childtype":[],"name":"北京大学","tag":[],"type":"科教文化服务;学校;高等院校","typecode":"141201","biz_type":[],"address":"颐和园路5号","location":"116.310905,39.992806","tel":"010-62752114","postcode":[],"website":"www.pku.edu.cn","email":[],"pcode":"110000","pname":"北京市","citycode":"010","cityname":"北京市","adcode":"110108","adname":"海淀区","importance":[],"shopid":[],"shopinfo":"0","poiweight":[],"gridcode":"5916729402","distance":[],"navi_poiid":"J50F001019_500002","entr_location":"116.315761,39.990097","business_area":[],"exit_location":[],"match":"0","recommend":"0","timestamp":"2019-07-01 00:20:51","alias":"PKU|北京大学燕园校区|北大","indoor_map":"0","indoor_data":{"cpid":[],"floor":[],"truefloor":[],"cmsid":[]},"groupbuy_num":"0","discount_num":"0","biz_ext":{"rating":[],"cost":[]},"event":[],"children":[],"photos":[{"title":[],"url":"http://store.is.autonavi.com/showpic/ad5f

{"status":"1","count":"895","info":"OK","infocode":"10000","suggestion":{"keywords":[],"cities":[]},"pois":[{"id":"B000A81FPJ","parent":[],"childtype":[],"name":"中央戏剧学院","tag":[],"type":"科教文化服务;学校;高等院校","typecode":"141201","biz_type":[],"address":"东棉花胡同39号","location":"116.404612,39.935973","tel":"010-64013958;010-56620312","postcode":[],"website":"www.chntheatre.edu.cn","email":[],"pcode":"110000","pname":"北京市","citycode":"010","cityname":"北京市","adcode":"110101","adname":"东城区","importance":[],"shopid":[],"shopinfo":"0","poiweight":[],"gridcode":"5916732201","distance":[],"navi_poiid":"J50F001020_1700","entr_location":"116.403983,39.935663","business_area":[],"exit_location":[],"match":"0","recommend":"0","timestamp":"2019-07-01 00:21:44","alias":[],"indoor_map":"0","indoor_data":{"cpid":[],"floor":[],"truefloor":[],"cmsid":[]},"groupbuy_num":"0","discount_num":"0","biz_ext":{"rating":[],"cost":[]},"event":[],"children":[],"photos":[{"title":[],"url":"http://store.is.autonavi.com/showp

{"status":"1","count":"895","info":"OK","infocode":"10000","suggestion":{"keywords":[],"cities":[]},"pois":[{"id":"B000A83XKJ","parent":[],"childtype":[],"name":"北京工业大学耿丹学院","tag":[],"type":"科教文化服务;学校;高等院校","typecode":"141201","biz_type":[],"address":"中富路牛山段3号","location":"116.651861,40.215624","tel":"010-60411788;010-60413297","postcode":[],"website":[],"email":[],"pcode":"110000","pname":"北京市","citycode":"010","cityname":"北京市","adcode":"110113","adname":"顺义区","importance":[],"shopid":[],"shopinfo":"0","poiweight":[],"gridcode":"6016255220","distance":[],"navi_poiid":"K50F046022_472","entr_location":"116.65504,40.215417","business_area":[],"exit_location":[],"match":"0","recommend":"0","timestamp":"2019-06-16 23:46:41","alias":[],"indoor_map":"0","indoor_data":{"cpid":[],"floor":[],"truefloor":[],"cmsid":[]},"groupbuy_num":"0","discount_num":"0","biz_ext":{"rating":[],"cost":[]},"event":[],"children":[],"photos":[{"title":"外景图","url":"http://store.is.autonavi.com/showpic/32b02604671ae

{"status":"1","count":"895","info":"OK","infocode":"10000","suggestion":{"keywords":[],"cities":[]},"pois":[{"id":"B000A81KWM","parent":[],"childtype":[],"name":"北京信息职业技术学院","tag":[],"type":"科教文化服务;学校;高等院校","typecode":"141201","biz_type":[],"address":"酒仙桥芳园西路5号","location":"116.48265,39.972991","tel":"010-64353978","postcode":[],"website":[],"email":[],"pcode":"110000","pname":"北京市","citycode":"010","cityname":"北京市","adcode":"110105","adname":"朝阳区","importance":[],"shopid":[],"shopinfo":"0","poiweight":[],"gridcode":"5916736821","distance":[],"navi_poiid":"J50F001020_456945","entr_location":"116.481728,39.974091","business_area":"酒仙桥","exit_location":[],"match":"0","recommend":"0","timestamp":"2019-06-27 21:17:27","alias":[],"indoor_map":"0","indoor_data":{"cpid":[],"floor":[],"truefloor":[],"cmsid":[]},"groupbuy_num":"0","discount_num":"0","biz_ext":{"rating":[],"cost":[]},"event":[],"children":[],"photos":[{"title":[],"url":"http://store.is.autonavi.com/showpic/45b247a7003a0d704d0745

{"status":"1","count":"895","info":"OK","infocode":"10000","suggestion":{"keywords":[],"cities":[]},"pois":[{"id":"B000A7P0Q2","parent":"B0FFFDC4Z1","childtype":"309","name":"北京工业大学通州校区","tag":[],"type":"科教文化服务;学校;高等院校","typecode":"141201","biz_type":[],"address":"潞苑南大街1145号","location":"116.668768,39.929539","tel":"010-52334033","postcode":[],"website":[],"email":[],"pcode":"110000","pname":"北京市","citycode":"010","cityname":"北京市","adcode":"110112","adname":"通州区","importance":[],"shopid":[],"shopinfo":"0","poiweight":[],"gridcode":"5916751311","distance":[],"navi_poiid":"J50F001022_209474","entr_location":"116.669111,39.927796","business_area":"永顺","exit_location":[],"match":"0","recommend":"0","timestamp":"2019-06-27 16:52:27","alias":"北工大通州","indoor_map":"0","indoor_data":{"cpid":[],"floor":[],"truefloor":[],"cmsid":[]},"groupbuy_num":"0","discount_num":"0","biz_ext":{"rating":[],"cost":[]},"event":[],"children":[],"photos":[{"title":"外景图","url":"http://store.is.autonavi.com/showpic/

...

{"status":"1","count":"398","info":"OK","infocode":"10000","suggestion":{"keywords":[],"cities":[]},"pois":[{"id":"B000A7OXET","parent":[],"childtype":[],"name":"北京市纺织服装职业技能培训学校","tag":[],"type":"科教文化服务;学校;职业技术学校","typecode":"141206","biz_type":[],"address":"新发地潘家庙56号","location":"116.348578,39.821883","tel":"010-87504827","postcode":[],"website":[],"email":[],"pcode":"110000","pname":"北京市","citycode":"010","cityname":"北京市","adcode":"110106","adname":"丰台区","importance":[],"shopid":[],"shopinfo":"0","poiweight":[],"gridcode":"5916528712","distance":[],"navi_poiid":"J50F003019_477800","entr_location":"116.348587,39.821794","business_area":"南苑","exit_location":[],"match":"0","recommend":"0","timestamp":"2019-06-27 12:42:29","alias":[],"indoor_map":"0","indoor_data":{"cpid":[],"floor":[],"truefloor":[],"cmsid":[]},"groupbuy_num":"0","discount_num":"0","biz_ext":{"rating":[],"cost":[]},"event":[],"children":[],"photos":[{"title":[],"url":"http://store.is.autonavi.com/showpic/ae2151a1427ecc3

{"status":"1","count":"398","info":"OK","infocode":"10000","suggestion":{"keywords":[],"cities":[]},"pois":[{"id":"B000A7ORII","parent":[],"childtype":[],"name":"北京市大兴区农业技术学校","tag":[],"type":"科教文化服务;学校;职业技术学校","typecode":"141206","biz_type":[],"address":"佟家场东巷","location":"116.319806,39.725279","tel":[],"postcode":[],"website":[],"email":[],"pcode":"110000","pname":"北京市","citycode":"010","cityname":"北京市","adcode":"110115","adname":"大兴区","importance":[],"shopid":[],"shopinfo":"0","poiweight":[],"gridcode":"5916427501","distance":[],"navi_poiid":"J50F004019_187273","entr_location":[],"business_area":[],"exit_location":[],"match":"0","recommend":"0","timestamp":"2019-06-29 10:50:47","alias":[],"indoor_map":"0","indoor_data":{"cpid":[],"floor":[],"truefloor":[],"cmsid":[]},"groupbuy_num":"0","discount_num":"0","biz_ext":{"rating":[],"cost":[]},"event":[],"children":[],"photos":[]},{"id":"B0FFFSFBLN","parent":[],"childtype":[],"name":"京城职业技能学校","tag":[],"type":"科教文化服务;学校;职业技术学校","typecode"

{"status":"1","count":"398","info":"OK","infocode":"10000","suggestion":{"keywords":[],"cities":[]},"pois":[{"id":"B0FFHG95OU","parent":[],"childtype":[],"name":"合济堂职业技能学校","tag":[],"type":"科教文化服务;学校;职业技术学校","typecode":"141206","biz_type":[],"address":"新中街177号宁静之都进门右拐直走","location":"116.851291,40.369368","tel":[],"postcode":[],"website":[],"email":[],"pcode":"110000","pname":"北京市","citycode":"010","cityname":"北京市","adcode":"110118","adname":"密云区","importance":[],"shopid":[],"shopinfo":"0","poiweight":[],"gridcode":"6016464800","distance":[],"navi_poiid":[],"entr_location":[],"business_area":[],"exit_location":[],"match":"0","recommend":"0","timestamp":[],"alias":[],"indoor_map":"0","indoor_data":{"cpid":[],"floor":[],"truefloor":[],"cmsid":[]},"groupbuy_num":"0","discount_num":"0","biz_ext":{"rating":[],"cost":[]},"event":[],"children":[],"photos":[{"title":[],"url":"http://store.is.autonavi.com/showpic/9de50a3622229628e4c481f7bf6dfdce"}]},{"id":"B0FFF0G008","parent":[],"childtype":[],

{"status":"1","count":"398","info":"OK","infocode":"10000","suggestion":{"keywords":[],"cities":[]},"pois":[{"id":"B000AA7KG7","parent":[],"childtype":[],"name":"阳光地带艺术学校-钢琴城第六分校","tag":[],"type":"科教文化服务;学校;职业技术学校|科教文化服务;培训机构;培训机构","typecode":"141206|141400","biz_type":[],"address":"奥运村街道万科星园东门","location":"116.414607,40.028417","tel":[],"postcode":[],"website":[],"email":[],"pcode":"110000","pname":"北京市","citycode":"010","cityname":"北京市","adcode":"110105","adname":"朝阳区","importance":[],"shopid":[],"shopinfo":"0","poiweight":[],"gridcode":"6016033310","distance":[],"navi_poiid":"K50F048020_503963","entr_location":[],"business_area":[],"exit_location":[],"match":"0","recommend":"0","timestamp":"2019-05-06 03:13:57","alias":[],"indoor_map":"0","indoor_data":{"cpid":[],"floor":[],"truefloor":[],"cmsid":[]},"groupbuy_num":"0","discount_num":"0","biz_ext":{"rating":[],"cost":[]},"event":[],"children":[],"photos":[{"title":[],"url":"http://store.is.autonavi.com/showpic/7f2a76d7cb6f2633504d08

{"status":"1","count":"0","info":"OK","infocode":"10000","suggestion":{"keywords":[],"cities":[]},"pois":[]}
分类：141206, 总的有390条数据
================分类：141206写入成功
